In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [2]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    # ... ENTER YOUR CODE HERE ...
    
    #Задаём y
    modx = np.linalg.norm(vec, ord=2)
    N = vec.shape[0]
    y = np.zeros((N,1), dtype=float)
    y[0] = modx
    
    #Вектор нормали
    u = (vec - y.T)/np.linalg.norm(vec - y.T)
    #u = np.asarray(u, dtype=float)
    
    H = np.eye(N) - 2*u.T@u
    return H@vec, H

In [3]:
v = np.array([1, 2, 3])
householder(v)

(array([ 3.74165739e+00,  0.00000000e+00, -1.11022302e-16]),
 array([[ 0.26726124,  0.53452248,  0.80178373],
        [ 0.53452248,  0.61007346, -0.5848898 ],
        [ 0.80178373, -0.5848898 ,  0.12266529]]))

Test your function using tests below:

In [4]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [5]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [6]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    #print(a1)
    # ... ENTER YOUR CODE HERE ...
    
    d = a1.T
    Q = np.eye(m, m)
    
    if m>=n:
        for i in range(n):
            d = a1.T
            b=d[i]
            e=np.zeros((m-i), dtype=float)
            for j in range(i, m):
                e[j-i]=b[j]
            
            y, H = householder(e)
            h=np.eye(m,m)
            for k in range(m-i):
                for s in range(m-i):
                    h[i+k, i+s]=H[k, s]
            a1=h@a1
            Q=Q@h
    else:
        for i in range(m-1):
            d = a1.T
            b=d[i]
            e=np.zeros((m-i), dtype=float)
            for j in range(i, m):
                e[j-i]=b[j]
            y, H = householder(e)
            h=np.eye(m,m)
            for k in range(m-i):
                for s in range(m-i):
                    h[i+k, i+s]=H[k, s]
            a1=h@a1
            Q=Q@h
    return Q, a1

In [7]:
a = np.random.random(size=(3, 5))
q, r = qr_decomp(a)
assert_allclose(np.dot(q, r), a)
print(q, "\n")
print(r, "\n")

[[ 0.68895988  0.25458894  0.67861532]
 [ 0.6338073  -0.66580838 -0.39368453]
 [ 0.35160004  0.70134419 -0.62007559]] 

[[ 6.90056275e-01  1.10187289e+00  9.76738001e-01  7.98430380e-02
   1.01272378e+00]
 [-2.95020853e-17  4.92288073e-01  8.58798596e-01  5.73611497e-02
  -1.67048317e-01]
 [-1.28808597e-16  2.85634746e-17  3.19908941e-02  2.24622377e-02
  -4.29874864e-01]] 



In [8]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [9]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [10]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

In [11]:
r

array([[ 1.40152769,  1.2514379 ,  0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        , -0.        ,  0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        , -0.        ]])

In [12]:
rr

array([[-1.40152769, -1.2514379 , -0.89523615],
       [ 0.        ,  0.84158252,  0.68447942],
       [ 0.        ,  0.        , -0.5496046 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [13]:
q

array([[ 0.13665049,  0.53601299, -0.09369752,  0.7697136 ,  0.30459557],
       [ 0.56035895,  0.0935397 , -0.53326881,  0.01839528, -0.62652547],
       [ 0.19725922,  0.65948912,  0.60068463, -0.32384673, -0.24589462],
       [ 0.62498418, -0.50418303,  0.52144688,  0.28453698,  0.04822969],
       [ 0.48765568,  0.12171264, -0.27224305, -0.47049398,  0.67223293]])

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [14]:
qq

array([[-0.13665049,  0.53601299,  0.09369752,  0.661619  , -0.49749149],
       [-0.56035895,  0.0935397 ,  0.53326881, -0.52477245, -0.34276292],
       [-0.19725922,  0.65948912, -0.60068463, -0.37879015,  0.14784752],
       [-0.62498418, -0.50418303, -0.52144688,  0.18967657, -0.21750907],
       [-0.48765568,  0.12171264,  0.27224305,  0.32774225,  0.75222783]])

In [15]:
q@r - qq@rr

array([[ 0.,  0., -0.],
       [ 0.,  0.,  0.],
       [ 0., -0., -0.],
       [-0.,  0., -0.],
       [-0., -0., -0.]])

*Enter your explanation here* (10% of the total grade, peer-graded)

In [16]:
Матрицы r и rr, q и qq отличаются знакамии у некоторых элементов, при этом qr и qqrr равны. 
Это означает либо то, что произошло чудо и нам повезло, либо то, что QR разложение неединственно.
Так как чудеса случаются редко, проверим вторую гипотизу. 

Возьмём матрицу -1
(-1)**2=1
Q(-1)(-1)R = QR = [Q*(-1)]*[(-1)*R]

Мы убедились, что для одной и той же матрицы существуют разные QR разложения.

SyntaxError: invalid syntax (<ipython-input-16-02a62e0860f5>, line 1)

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [75]:
if m>=n:
    for i in range(n):
        A = a1.T
        b=A[i]
        e=np.zeros((m-i), dtype=float)
        for j in range(i, m):
            e[j-i]=b[j]
            u=np.zeros((m-i,), dtype=float)
            y=np.zeros((m-i, 1), dtype=float)
            u=(e-y.T)/np.linalg.norm(e-y.T)
            U=np.zeros((m,), dtype=float)
            for k in range(m-i+1):
                U[k+i]=u[k]
    else:
        for i in range(m-1):
        A = a1.T
        b=A[i]
        e=np.zeros((m-i), dtype=float)
        for j in range(i, m):
            e[j-i]=b[j]
            u=np.zeros((m-i,), dtype=float)
            y=np.zeros((m-i, 1), dtype=float)
            u=(e-y.T)/np.linalg.norm(e-y.T)
            U=np.zeros((m,), dtype=float)
            for k in range(m-i+1):
                U[k+i]=u[k]

A [[0.089692   0.78090572 0.10216144 0.98267867 0.27461371]
 [0.67159824 0.17946842 0.09209561 0.25561229 0.02551328]
 [0.54002837 0.8303011  0.67032625 0.24396706 0.89009341]]

b [0.089692   0.78090572 0.10216144 0.98267867 0.27461371]
[0.089692   0.78090572 0.10216144 0.98267867 0.27461371]
A [[0.089692   0.78090572 0.10216144 0.98267867 0.27461371]
 [0.67159824 0.17946842 0.09209561 0.25561229 0.02551328]
 [0.54002837 0.8303011  0.67032625 0.24396706 0.89009341]]

b [0.67159824 0.17946842 0.09209561 0.25561229 0.02551328]
[0.17946842 0.09209561 0.25561229 0.02551328]
A [[0.089692   0.78090572 0.10216144 0.98267867 0.27461371]
 [0.67159824 0.17946842 0.09209561 0.25561229 0.02551328]
 [0.54002837 0.8303011  0.67032625 0.24396706 0.89009341]]

b [0.54002837 0.8303011  0.67032625 0.24396706 0.89009341]
[0.67032625 0.24396706 0.89009341]


In [76]:
'''rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)'''

'rndm = np.random.RandomState(1234)\na = rndm.uniform(size=(5, 3))\nq, r = qr_decomp(a)'